In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device", device)
from unsloth.chat_templates import get_chat_template
from datasets import load_dataset
import unsloth


from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
from transformers import Trainer, TrainingArguments
from trl import SFTTrainer
import re
from join_string import join_strings
from mcpu5 import simulate_dual_core

device cuda
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Core 1 Execution Time: 26 cycles
Core 2 Execution Time: 9 cycles


In [2]:
def code_extractor(inputs):
    pattern = r'"(.*?)"'
    # Using re.DOTALL to match across multiple lines
    matches = re.findall(pattern, inputs, re.DOTALL)
    
    # Strip whitespace from each match
    code_blocks = [match.strip() for match in matches]
    #print(code_blocks)
    return code_blocks

In [3]:
def message2code(message):
    inputs = tokenizer.apply_chat_template(
        message,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_dict = True,
        return_tensors = "pt",
    ).to("cuda")
    outputs = model.generate(input_ids = inputs["input_ids"],
                        attention_mask = inputs["attention_mask"],
                        max_new_tokens = 256,
                        use_cache = True)
    outputs = tokenizer.batch_decode(outputs)[0]
    print("outputs", outputs)
    out = code_extractor(outputs)
    return out

In [4]:
def make_random_code():
    message_init = [
    {"from": "human", "value": f"""I have a cpu simulator with registers R1 up to R10, and that takes assembly instructions STORE, LOAD, ADD, MUL as input. \n
    Here is an example of a list of instructions in this language:
    \n DIV R5, R6\n SUB R7, R8 \n LOAD R9, 30\nADD R9, R10\n
    Can you write a random list of instructions each presented inside quotation marks please?
    Please make a list between 1 and 20 instructions.
        """},
    ]
    return message2code(message_init)

In [5]:
class History:
    def __init__(self, max_size = 100):
        self.max_size = max_size
        self.memory_program = []
        self.memory_signature = {"core1_exec_time":[],
                                 "core2_exec_time":[]}
        self.min_exec_time_core1 = None
        self.max_exec_time_core1 = None
        self.min_exec_time_core2 = None
        self.max_exec_time_core2 = None
    def store(self,sample:dict[list]):
        for j in range(len(sample["program"])):
            self.memory_program.append(sample["program"][j])
            self.memory_signature["core1_exec_time"].append(sample["signature"][j]["core1_exec_time"])
            self.memory_signature["core2_exec_time"].append(sample["signature"][j]["core2_exec_time"])
        self.eviction()
        
    def eviction(self):
        if len(self.memory_program)>self.max_size:
            self.memory_program = self.memory_program[-self.max_size:]
            self.memory_signature = self.memory_signature[-self.max_size:]
    def select_closest_code(self,signature: dict)->dict:
        min_distance = 0
        idx = 0
        for j,signature_buffer in enumerate(self.memory_signature):
            dist = 0
            for k in signature_buffer.keys():
                dist += (signature_buffer[k] - signature[k])**2
            if min_distance:
                if dist<min_distance:
                    min_distance= dist
                    idx = j
            else:
                min_distance = dist
                idx = j
        return {"code": self.code_list[idx] ,"signature": self.signature_list[idx]}

In [6]:
class GoalGenerator:
    def __init__(self):
        pass

In [7]:
class OptimizationPolicy:
    """
    Selects a parameter based on a chosen goal and the history.
    Takes the code corresponding to the closest signature to the desired goal signature
    """
    def __init__(self):    
        pass
    def __call__(goal:dict[list],H:History):
        closest_code = H.select_closest_code(goal) #most promising sample from the history
        output = self.light_code_mutation(closest_code) #expansion strategie: small random mutation
    def light_code_mutation(self,program:list[str]):
        messages = [
        {"from": "human", "value": f"""I have a cpu simulator with registers R1 up to R10, and that takes assembly instructions STORE, LOAD, ADD, MUL as input. \n
        Here is an example of a list of instructions in this language:
        \n DIV R5, R6\n SUB R7, R8 \n LOAD R9, 30\nADD R9, R10\n
        
        A mutation of a list of instructions consists in inserting, deleting or replacing a few instruction in program. For instance, here is a mutation of the list above. I added a the instruction LOAD in the fist line and I have replaced the last instruction by an instruction STORE.
        
        \nLOAD R4, 30\n DIV R5, R6\n SUB R7, R8 \n LOAD R9, 30\nSTORE R1, 20\n
        
        Can you perform a single light mutation based on the following list of instructions bellow?     
        Your answer has to be in the following format:
        
            Mutated list of intructions inside quotation marks
            Reflexion on how you mutate it
        List of instructions:
        {join_strings(program)}
            """},
        ]
        return message2code(messages)

In [8]:
class IMGEP:
    def __init__(self,N:int, N_init:int,H:History, G:GoalGenerator, Pi:OptimizationPolicy):
        """
        N: int. The experimental budget
        """
        self.N = N
        self.H = H
        self.G = G
        self.N_init = N_init
        self.Pi = Pi
    def __call__(self):
        for i in range(self.N):
            if i<self.N_init:
                #Initial random iterations
                core1_code = make_random_code()
            else:
                break
                #Sample target goal
                goal_code = self.G(self.H)
                core1_code = self.Pi(goal_code,self.H)
                 
            core1_exec_time, core2_exec_time = simulate_dual_core(
            core1_code = core1_code,
            core2_code =["MUL R3, R4",
                        "STORE R1, 20",
                        "MOV R5, R6",
                        "LOAD R1, 10",
                        "ADD R1, R2",
                        "MUL R3, R4",])
            self.H.store({"program":[core1_code],
                         "signature": [{"core1_exec_time":core1_exec_time,
                                        "core2_exec_time": core2_exec_time}]})

In [9]:
max_seq_length = 512# Can increase for longer reasoning traces
try:
    model, tokenizer = FastLanguageModel.from_pretrained(
    #model_name = "mistralai/Mistral-7B-v0.1",
    model_name = "meta-llama/meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    #load_in_4bit = True, # False for LoRA 16bit
    #fast_inference = True, # Enable vLLM fast inference
    #max_lora_rank = lora_rank,
    #gpu_memory_utilization = 0.1, # Reduce if out of memory
    )


    tokenizer = get_chat_template(
        tokenizer,
        chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
        mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
        map_eos_token = True, # Maps <|im_end|> to </s> instead
    )

    model = FastLanguageModel.get_peft_model(
        model,
    )
except:
    print("Maybe the model is already specified")

==((====))==  Unsloth 2025.3.9: Fast Llama patching. Transformers: 4.49.0. vLLM: 0.7.3.
   \\   /|    NVIDIA RTX 2000 Ada Generation Laptop GPU. Num GPUs = 1. Max memory: 7.754 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Will map <|im_end|> to EOS = <|eot_id|>.
Unsloth 2025.3.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [10]:
H = History(max_size = 100)

In [11]:
Pi = OptimizationPolicy()

In [12]:
G = GoalGenerator()

In [13]:
imgep = IMGEP(N=3, N_init = 3,H=H, G=G, Pi=Pi)

In [14]:
imgep()

outputs <|im_start|>user
I have a cpu simulator with registers R1 up to R10, and that takes assembly instructions STORE, LOAD, ADD, MUL as input. 

    Here is an example of a list of instructions in this language:
    
 DIV R5, R6
 SUB R7, R8 
 LOAD R9, 30
ADD R9, R10

    Can you write a random list of instructions each presented inside quotation marks please?
    Please make a list between 1 and 20 instructions.
        <|im_end|>
<|im_start|>assistant
Here is a list of 15 random instructions:

"LOAD R2, 50"
"STORE R3, R1"
"MUL R4, R5"
"ADD R6, R7"
"LOAD R8, 20"
"SUB R9, R10"
"STORE R1, R2"
"DIV R3, R4"
"LOAD R5, 10"
"ADD R6, R8"
"MUL R7, R9"
"LOAD R10, 40"
"STORE R4, R6"
"SUB R1, R3"
"LOAD R9, 15"<|im_end|>
outputs <|im_start|>user
I have a cpu simulator with registers R1 up to R10, and that takes assembly instructions STORE, LOAD, ADD, MUL as input. 

    Here is an example of a list of instructions in this language:
    
 DIV R5, R6
 SUB R7, R8 
 LOAD R9, 30
ADD R9, R10

    Can 

In [15]:
H.memory_signature

{'core1_exec_time': [32, 29, 21], 'core2_exec_time': [14, 12, 14]}

In [17]:
program1 = ["MOV R1, R2",
    "LOAD R1, 10",
    "ADD R1, R2",
    "MUL R3, R4",
    "STORE R1, 20",
    "MOV R5, R6",
    "LOAD R1, 10",
    "ADD R1, R2",
    "MUL R3, R4",
    "STORE R1, 20",
    "DIV R5, R6",
    "SUB R7, R8",
    "LOAD R9, 30",
    "ADD R9, R10"]

messages = [
    {"from": "human", "value": f"""I have a cpu simulator with registers R1 up to R10, and that takes assembly instructions STORE, LOAD, ADD, MUL as input. \n
Here is an example of a list of instructions in this language:
\n DIV R5, R6\n SUB R7, R8 \n LOAD R9, 30\nADD R9, R10\n

A mutation of a list of instructions consists in inserting, deleting or replacing a few instruction in program. For instance, here is a mutation of the list above. I added a the instruction LOAD in the fist line and I have replaced the last instruction by an instruction STORE.

\nLOAD R4, 30\n DIV R5, R6\n SUB R7, R8 \n LOAD R9, 30\nSTORE R1, 20\n

Can you perform a single light mutation based on the following list of instructions bellow?     
Your answer has to be in the following format:

    Mutated list of intructions inside quotation marks
    Reflexion on how you mutate it
List of instructions:
{join_strings(program1)}
    """},
]